# Base Video Pipeline

Load overlay

In [1]:
from pynq import Overlay
from pynq.lib.video import *

base = Overlay('/home/xilinx/jupyter_notebooks/contrast/base.bit')
base.download()

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

Configure HDMI

In [2]:
hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)

hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)

In [3]:
from HAL import *

contrast = Contrast(0x4000_0000)
sharpen = Sharpen(0x4001_0000)

In [11]:
contrast.set_scaler(0.5)

32768


In [15]:
contrast.enable()

In [12]:
contrast.disable()

In [16]:
sharpen.enable()

In [17]:
sharpen.disable()

Don't forget to close the HDMI connection!

In [65]:
hdmi_out.close()
hdmi_in.close()